# Load data

In [ ]:
import json
with open("output/selected_dialogues.json", "r") as f:
    data = json.load(f)
    print(data)

In [4]:
# data_sample is the data in barem so remove it in data
data_sample = []
data_inference = []

for dialogue in data['dialogues']:
    if (dialogue['dialogue_id'] in [335, 25, 26]):
        data_sample.append(dialogue)
    else:
        data_inference.append(dialogue)
        
print("Number of sample dialogues:", len(data_sample))
print("Number of inference dialogues:", len(data_inference))

Number of sample dialogues: 3
Number of inference dialogues: 38


In [ ]:
import copy

def split_data_label(data):
    print(f"Split_data_label for {len(data)} records.")
    preprocess_dialogue_data = copy.deepcopy(data)

    # score each dialogue
    average_scores = []
    average_score_100 = []
    overall_scores = []
    for dialogue in preprocess_dialogue_data:
        average_scores.append({"dialogue_id": dialogue['dialogue_id'], "average_scores": dialogue['average_score']})
        average_score_100.append({"dialogue_id": dialogue['dialogue_id'], "average_score_100": dialogue['average_score_100']})
        overall_scores.append({"dialogue_id": dialogue['dialogue_id'], "overall_scores": dialogue['overall_scores']})

    # Preprocess into data and label (scores)
    
    for dialogue in preprocess_dialogue_data:
        for turn in dialogue['turns']:
            # remove the "scores" field
            if 'scores' in turn:
                del turn['scores']
            if 'intent' in turn:
                del turn['intent']
            
    data_inference = []
    for dialogue in preprocess_dialogue_data:
        dialogue_text = ""
        for turn in dialogue['turns']:
            dialogue_text += turn['speaker'] + ": " + turn['text'] + "\n"
        data_inference.append({"dialogue_id": dialogue['dialogue_id'], "text": dialogue_text})
        
    return data_inference, average_scores, average_score_100, overall_scores

In [6]:
data_inference, average_scores, average_score_100, overall_scores = split_data_label(data_inference)

Split_data_label for 38 records.


# Load result

In [35]:
result = [{'dialogue_id': 1, 'average_score_100': 50.5, 'model_score': 35},
 {'dialogue_id': 3, 'average_score_100': 50.5, 'model_score': 38},
 {'dialogue_id': 2, 'average_score_100': 56.69, 'model_score': 62},
 {'dialogue_id': 7, 'average_score_100': 56.69, 'model_score': 64},
 {'dialogue_id': 9, 'average_score_100': 58.75, 'model_score': None},
 {'dialogue_id': 15, 'average_score_100': 58.75, 'model_score': 18},
 {'dialogue_id': 10, 'average_score_100': 67.0, 'model_score': 68},
 {'dialogue_id': 20, 'average_score_100': 67.0, 'model_score': 48},
 {'dialogue_id': 13, 'average_score_100': 62.88, 'model_score': 54},
 {'dialogue_id': 29, 'average_score_100': 62.88, 'model_score': None},
 {'dialogue_id': 14, 'average_score_100': 69.06, 'model_score': 79},
 {'dialogue_id': 115, 'average_score_100': 69.06, 'model_score': 34},
 {'dialogue_id': 16, 'average_score_100': 44.31, 'model_score': 68},
 {'dialogue_id': 41, 'average_score_100': 44.31, 'model_score': 68},
 {'dialogue_id': 18, 'average_score_100': 31.94, 'model_score': 38},
 {'dialogue_id': 70, 'average_score_100': 31.94, 'model_score': None},
 {'dialogue_id': 24, 'average_score_100': 65.35, 'model_score': 28},
 {'dialogue_id': 326, 'average_score_100': 65.35, 'model_score': 55},
 {'dialogue_id': 136, 'average_score_100': 75.25, 'model_score': 18},
 {'dialogue_id': 202, 'average_score_100': 25.75, 'model_score': None}
 ]

In [ ]:
import pandas as pd
result_df = pd.DataFrame(result)
df = pd.DataFrame(average_scores)

In [ ]:
result_df = result_df.join(df, on='dialogue_id', lsuffix='_left', rsuffix='_right')
result_df = result_df[['dialogue_id', 'average_scores', 'model_score']]
result_df['model_score_5'] = result_df['model_score'] / 20
result_df.drop(columns=['model_score'], inplace=True)

result_df = result_df.dropna()

# Evaluate

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_all_metrics(y_true, y_pred):
    """
    Calculates MAE, MSE, RMSE, and R-squared and returns them in a dictionary.
    
    Args:
        y_true (list or np.array): True Average Scores (in dataset).
        y_pred (list or np.array): Predicted Average Scores (model generated).
    """
    mae = round(mean_absolute_error(y_true, y_pred), 4)
    mse = round(mean_squared_error(y_true, y_pred), 4)
    rmse = float(round(np.sqrt(mse), 4))
    r2 = round(r2_score(y_true, y_pred), 4)
    
    metrics = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }
    
    return metrics

In [32]:
metrics_list = []

In [ ]:
# Vd đã chạy 3 lần (chạy 3 lần ô này)
metrics = calculate_all_metrics(result_df['average_scores'], result_df['model_score_5'])
metrics_list.append(metrics)

In [41]:
all_results

{'mean': {'MAE': 0.8642, 'MSE': 1.173, 'RMSE': 1.0831, 'R2': -2.5142},
 'sd': {'MAE': 0.0, 'MSE': 0.0, 'RMSE': 0.0, 'R2': 0.0}}